<h3> Data Cleaning </h3>

In [1]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
delegates = pd.read_csv("Data/Delegates/cleaned/constitutional_convention_delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)
state_delegates = pd.read_csv("Data/Delegates/cleaned/State_Delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)

In [3]:
final_matching = pd.read_csv('Data/final_matching.csv').drop(['Unnamed: 0', 'Scores'], axis = 1)

In [4]:
spec_names = final_matching[final_matching['Delegates'] != final_matching['Loan Matches']]

# Delegate Asset Database

In [5]:
delegates_assets = delegates[['full name 1', 'full name 2', 'sign']]
state_delegate_assets = state_delegates[['full name 1', 'State', 'Vote']]
state_delegate_assets.rename(columns = {"Vote":"sign", "State":"delegate_state"}, inplace = True)
total_assets = pd.concat([delegates_assets, state_delegate_assets]).reset_index(drop = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [6]:
#amending existing cases

In [7]:
total_assets['full name 2'] = total_assets['full name 1']
total_assets_full = total_assets

In [8]:
merge_1 = pd.merge(total_assets_full.drop('full name 2', axis = 1), 
                   spec_names, left_on = 'full name 1', 
                   right_on = 'Delegates').drop('full name 1', axis = 1)
merge_2 = pd.merge(total_assets_full.drop('full name 1', axis = 1), 
                   spec_names, left_on = 'full name 2', 
                   right_on = 'Delegates').drop('full name 2', axis = 1)
merged = pd.concat([merge_1, merge_2]).drop_duplicates()

In [9]:
total_assets_full.columns = ['delegate_state', 'delegate name', 'match name', 'sign']
merged.columns =  ['delegate_state', 'sign',  'delegate name', 'match name']

In [10]:
total_assets_final = pd.concat([total_assets_full, merged])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


### Full Name - First/Last Name Mapping

In [11]:
state_firstlast = state_delegates[['full name 1', 'First Name', 'Last Name']].rename(columns = {'full name 1': 'Full Name'})
const_firstlast = delegates[['full name 1', 'first name', 'last name']].rename(columns = {'first name':'First Name',
                                                                                          'last name':'Last Name',
                                                                                          'full name 1': 'Full Name'})
cum_firstlast = pd.concat([state_firstlast, const_firstlast])

# Connecticut Assumed Debt Dataset Matching

In [12]:
#prepare loan dataset
CT_ASD = pd.read_excel("Data/Post1790/CT/CT_post1790_ASD_ledger.xlsx", header = 13, usecols = 'H, I, N, O, X, Y, AD, AE, AN, AO, AT, AU')
CT_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
CT_ASD['full name 1'] = CT_ASD['First Name'] + " " + CT_ASD['Last Name']
CT_ASD['full name 2'] = CT_ASD['First Name.1'] + " " + CT_ASD['Last Name.1']
CT_ASD['full name 3'] = CT_ASD['First Name.2'] + " " + CT_ASD['Last Name.2']

In [13]:
CT_ASD['index'] = CT_ASD.index

CT_ASD_f1 = CT_ASD[['full name 1', '6p_Dollar','6p_Cents',]]
CT_ASD_f2 = CT_ASD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
CT_ASD_f3 = CT_ASD[['full name 3', '3p_Dollar', '3p_Cents']]

In [387]:
CT_ASD_6p = pd.merge(total_assets_final, CT_ASD_f1[CT_ASD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
CT_ASD_6p_def = pd.merge(total_assets_final, CT_ASD_f2[CT_ASD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
CT_ASD_3p = pd.merge(total_assets_final, CT_ASD_f3[CT_ASD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [388]:
CT_ASD_agg = pd.merge(CT_ASD_6p, CT_ASD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
CT_ASD_agg = pd.merge(CT_ASD_agg, CT_ASD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
CT_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Roger Sherman,NaN,yes,3435.0,91.0,1717.0,96.0,NaN,NaN
1,Roger Sherman,CT,yes,3435.0,91.0,1717.0,96.0,NaN,NaN
2,Oliver Ellsworth,NaN,no,2660.0,98.0,1330.0,50.0,1995.0,75.0
3,Oliver Ellsworth,CT,yes,2660.0,98.0,1330.0,50.0,1995.0,75.0
4,John Dickinson,NaN,yes,3227.0,29.0,1613.0,65.0,2420.0,47.0


# Maryland Assumed Debt Dataset Matching

In [389]:
#prepare loan dataset
MD_ASD = pd.read_excel("Data/Post1790/MD/MD_post1790_ASD.xlsx", header = 11, usecols = 'G, H, L, M, U, V, Z, AA, AI, AJ, AN, AO')
MD_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
MD_ASD['full name 1'] = MD_ASD['First Name'] + " " + MD_ASD['Last Name']
MD_ASD['full name 2'] = MD_ASD['First Name.1'] + " " + MD_ASD['Last Name.1']
MD_ASD['full name 3'] = MD_ASD['First Name.2'] + " " + MD_ASD['Last Name.2']

In [390]:
MD_ASD['index'] = MD_ASD.index

MD_ASD_f1 = MD_ASD[['full name 1', '6p_Dollar','6p_Cents',]]
MD_ASD_f2 = MD_ASD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
MD_ASD_f3 = MD_ASD[['full name 3', '3p_Dollar', '3p_Cents']]

In [391]:
MD_ASD_6p = pd.merge(total_assets_final, MD_ASD_f1[MD_ASD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
MD_ASD_6p_def = pd.merge(total_assets_final, MD_ASD_f2[MD_ASD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
MD_ASD_3p = pd.merge(total_assets_final, MD_ASD_f3[MD_ASD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [392]:
MD_ASD_agg = pd.merge(MD_ASD_6p, MD_ASD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
MD_ASD_agg = pd.merge(MD_ASD_agg, MD_ASD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
MD_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,William Moore,DE,yes,2726.0,99.0,1363.0,50.0,2045.0,93.0
1,George Plater,MD,yes,127.0,86.0,NaN,NaN,95.0,91.0
2,Nicholas Carroll,MD,yes,685.0,96.0,342.0,98.0,514.0,48.0
3,George Gale,MD,yes,1894.0,21.0,947.0,11.0,1420.0,66.0
4,James Williams,MA,no,8645.0,13.0,4322.0,58.0,6483.0,87.0


In [393]:
cumulative_ASD = pd.concat([CT_ASD_agg, MD_ASD_agg])

# North Carolina Assumed Debt Dataset Matching

In [394]:
#prepare loan dataset
NC_ASD = pd.read_excel("Data/Post1790/NC/T695_R3_NC_ASD.xlsx", header = 10, usecols = 'H, I, P, Q, R, S, T, U')
NC_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NC_ASD['full name 1'] = NC_ASD['First Name'] + " " + NC_ASD['Last Name']

In [395]:
NC_ASD_f = NC_ASD[['full name 1', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']]

In [396]:
NC_ASD_agg = pd.merge(total_assets_final, NC_ASD_f[NC_ASD_f['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1', 'match name'], axis = 1)

In [397]:
NC_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,William Blount,NaN,yes,61.0,492.66,61.0,246.32,61.0,369.49
1,William Blount,NC,yes,61.0,492.66,61.0,246.32,61.0,369.49
2,William Blount,NC,yes,61.0,492.66,61.0,246.32,61.0,369.49
3,Hugh Williamson,NaN,yes,61.0,1086.60,61.0,543.30,61.0,814.94
4,Hugh Williamson,NC,yes,61.0,1086.60,61.0,543.30,61.0,814.94


In [398]:
cumulative_ASD = pd.concat([NC_ASD_agg, cumulative_ASD])

# New Hampshire Assumed Debt Dataset Matching

In [399]:
#prepare loan dataset
NH_ASD = pd.read_excel("Data/Post1790/NH/T652_New_Hampshire_ASD.xlsx", header = 11, usecols = 'G, H, M, N, V, W, AB, AC, AK, AL, AM, AN')
NH_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
NH_ASD['full name 1'] = NH_ASD['First Name'] + " " + NH_ASD['Last Name']
NH_ASD['full name 2'] = NH_ASD['First Name.1'] + " " + NH_ASD['Last Name.1']
NH_ASD['full name 3'] = NH_ASD['First Name.2'] + " " + NH_ASD['Last Name.2']

In [400]:
NH_ASD['index'] = NH_ASD.index

NH_ASD_f1 = NH_ASD[['full name 1', '6p_Dollar','6p_Cents',]]
NH_ASD_f2 = NH_ASD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
NH_ASD_f3 = NH_ASD[['full name 3', '3p_Dollar', '3p_Cents']]

In [401]:
NH_ASD_6p = pd.merge(total_assets_final, NH_ASD_f1[NH_ASD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
NH_ASD_6p_def = pd.merge(total_assets_final, NH_ASD_f2[NH_ASD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
NH_ASD_3p = pd.merge(total_assets_final, NH_ASD_f3[NH_ASD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [402]:
NH_ASD_agg = pd.merge(NH_ASD_6p, NH_ASD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
NH_ASD_agg = pd.merge(NH_ASD_agg, NH_ASD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
NH_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,John Langdon,NaN,yes,959.0,25.0,479.0,63.0,719.4375,NaN
1,John Langdon,NaN,yes,959.0,25.0,479.0,63.0,124.9425,NaN
2,John Langdon,NaN,yes,959.0,25.0,479.0,63.0,129.8700,NaN
3,John Langdon,NaN,yes,959.0,25.0,83.0,30.0,719.4375,NaN
4,John Langdon,NaN,yes,959.0,25.0,83.0,30.0,124.9425,NaN


In [403]:
cumulative_ASD = pd.concat([NH_ASD_agg, cumulative_ASD])

# New York Assumed Debt Dataset Matching

In [404]:
#prepare loan dataset
NY_ASD = pd.read_excel("Data/Post1790/NY/NY_1790_ASD.xlsx", header = 11, usecols = 'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS')
NY_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
NY_ASD['full name 1'] = NY_ASD['First Name'] + " " + NY_ASD['Last Name']
NY_ASD['full name 2'] = NY_ASD['First Name.1'] + " " + NY_ASD['Last Name.1']
NY_ASD['full name 3'] = NY_ASD['First Name.2'] + " " + NY_ASD['Last Name.2']

In [405]:
NY_ASD['index'] = NY_ASD.index

NY_ASD_f1 = NY_ASD[['full name 1', '6p_Dollar','6p_Cents',]]
NY_ASD_f2 = NY_ASD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
NY_ASD_f3 = NY_ASD[['full name 3', '3p_Dollar', '3p_Cents']]

In [406]:
NY_ASD_6p = pd.merge(total_assets_final, NY_ASD_f1[NY_ASD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
NY_ASD_6p_def = pd.merge(total_assets_final, NY_ASD_f2[NY_ASD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
NY_ASD_3p = pd.merge(total_assets_final, NY_ASD_f3[NY_ASD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [407]:
NY_ASD_agg = pd.merge(NY_ASD_6p, NY_ASD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
NY_ASD_agg = pd.merge(NY_ASD_agg, NY_ASD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
NY_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Rufus King,NaN,yes,1357,NaN,678.0,50,1017.0,74.0
1,Rufus King,NaN,yes,1357,NaN,678.0,50,421.0,60.0
2,Rufus King,NaN,yes,281,0.0,678.0,50,1017.0,74.0
3,Rufus King,NaN,yes,281,0.0,678.0,50,421.0,60.0
4,Rufus King,MA,yes,1357,NaN,678.0,50,1017.0,74.0


In [408]:
cumulative_ASD = pd.concat([NY_ASD_agg, cumulative_ASD])

# Rhode Island Assumed Debt Dataset Matching

In [409]:
#prepare loan dataset
RI_ASD = pd.read_excel("Data/Post1790/RI/T653_Rhode_Island_ASD.xlsx", header = 11, usecols = 'H, I, N, O, X, Y, AD, AE, AN, AO, AT, AU')
RI_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
RI_ASD['full name 1'] = RI_ASD['First Name'] + " " + RI_ASD['Last Name']
RI_ASD['full name 2'] = RI_ASD['First Name.1'] + " " + RI_ASD['Last Name.1']
RI_ASD['full name 3'] = RI_ASD['First Name.2'] + " " + RI_ASD['Last Name.2']

In [410]:
RI_ASD['index'] = RI_ASD.index

RI_ASD_f1 = RI_ASD[['full name 1', '6p_Dollar','6p_Cents',]]
RI_ASD_f2 = RI_ASD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
RI_ASD_f3 = RI_ASD[['full name 3', '3p_Dollar', '3p_Cents']]

In [411]:
RI_ASD_6p = pd.merge(total_assets_final, RI_ASD_f1[RI_ASD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
RI_ASD_6p_def = pd.merge(total_assets_final, RI_ASD_f2[RI_ASD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
RI_ASD_3p = pd.merge(total_assets_final, RI_ASD_f3[RI_ASD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [412]:
RI_ASD_agg = pd.merge(RI_ASD_6p, RI_ASD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
RI_ASD_agg = pd.merge(RI_ASD_agg, RI_ASD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
RI_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Benjamin Hall,MD,yes,405.0,2.0,202.0,50.0,303.0,76.0
1,Thomas Allen,VA,no,12.0,64.0,6.0,32.0,9.0,47.0
2,Allen Thomas,RI,yes,12.0,64.0,6.0,32.0,9.0,47.0
3,John Smith,NY,yes,2271.0,93.0,1135.0,96.0,1703.0,94.0
4,John K. Smith,MA,yes,2271.0,93.0,1135.0,96.0,1703.0,94.0


In [413]:
cumulative_ASD = pd.concat([RI_ASD_agg, cumulative_ASD])

# South Carolina Assumed Debt Dataset Matching

In [414]:
#prepare loan dataset
SC_ASD = pd.read_excel("Data/Post1790/SC/Post_1790_South_Carolina_ASD_transfers_removed.xlsx", header = 11, usecols = 'D, E, M, N, O')
SC_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_def_Dollar','3p_Dollar']
SC_ASD['full name 1'] = SC_ASD['First Name'] + " " + SC_ASD['Last Name']

In [415]:
SC_ASD['6p_Cents'] = SC_ASD['6p_Dollar'] % 1
SC_ASD['6p_Dollar'] = np.floor(SC_ASD['6p_Dollar'])
SC_ASD['6p_def_Cents'] = SC_ASD['6p_def_Dollar'] % 1
SC_ASD['6p_def_Dollar'] = np.floor(SC_ASD['6p_def_Dollar'])
SC_ASD['3p_Cents'] = SC_ASD['3p_Dollar'] % 1
SC_ASD['3p_Dollar'] = np.floor(SC_ASD['3p_Dollar'])

In [416]:
SC_ASD_f = SC_ASD[['full name 1', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']]

In [417]:
SC_ASD_agg = pd.merge(total_assets_final, SC_ASD_f[SC_ASD_f['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1', 'match name'], axis = 1)

In [418]:
SC_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Robert Morris,NaN,yes,1767.0,0.875556,883.0,0.937778,1325.0,0.906667
1,Thomas Fitzsimons,NaN,yes,1010.0,0.933333,505.0,0.466667,758.0,0.200000
2,John Rutledge,NaN,yes,579.0,0.280000,289.0,0.630000,434.0,0.450000
3,John Rutledge,SC,yes,579.0,0.280000,289.0,0.630000,434.0,0.450000
4,Daniel Hall,CT,no,2072.0,0.160000,1036.0,0.700000,1554.0,0.110000


In [419]:
cumulative_ASD = pd.concat([SC_ASD_agg, cumulative_ASD])

# Virginia Assumed Debt Dataset Matching

In [420]:
#prepare loan dataset
VA_ASD = pd.read_excel("Data/Post1790/VA/VA_ASD.xlsx", header = 11, usecols = 'D, E, N, O, U, V, AE, AF, AL, AM, AW, AX')
VA_ASD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
VA_ASD['full name 1'] = VA_ASD['First Name'] + " " + VA_ASD['Last Name']
VA_ASD['full name 2'] = VA_ASD['First Name.1'] + " " + VA_ASD['Last Name.1']
VA_ASD['full name 3'] = VA_ASD['First Name.2'] + " " + VA_ASD['Last Name.2']

In [421]:
VA_ASD['index'] = VA_ASD.index

VA_ASD_f1 = VA_ASD[['full name 1', '6p_Dollar','6p_Cents',]]
VA_ASD_f2 = VA_ASD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
VA_ASD_f3 = VA_ASD[['full name 3', '3p_Dollar', '3p_Cents']]

In [422]:
VA_ASD_6p = pd.merge(total_assets_final, VA_ASD_f1[VA_ASD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
VA_ASD_6p_def = pd.merge(total_assets_final, VA_ASD_f2[VA_ASD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
VA_ASD_3p = pd.merge(total_assets_final, VA_ASD_f3[VA_ASD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [423]:
VA_ASD_agg = pd.merge(VA_ASD_6p, VA_ASD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
VA_ASD_agg = pd.merge(VA_ASD_agg, VA_ASD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
VA_ASD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,John Blair,NaN,yes,4229.0,97.0,2114.0,99.0,3172.0,48.0
1,John Blair,NaN,yes,4229.0,97.0,2114.0,99.0,642.0,3.0
2,John Blair,NaN,yes,4229.0,97.0,428.0,2.0,3172.0,48.0
3,John Blair,NaN,yes,4229.0,97.0,428.0,2.0,642.0,3.0
4,John Blair,NaN,yes,856.0,4.0,2114.0,99.0,3172.0,48.0


In [424]:
cumulative_ASD = pd.concat([VA_ASD_agg, cumulative_ASD])

# Assembling Cumulative Assumed State Debt File

In [425]:
cumulative_ASD['delegate_state'] = cumulative_ASD['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

In [426]:
cumulative_ASD_assets = pd.merge(cum_firstlast, cumulative_ASD, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()

In [427]:
cumulative_ASD_assets = cumulative_ASD_assets.rename(columns = {'6p_Dollar':'ASD_6p_Dollar', '6p_Cents':'ASD_6p_Cents','6p_def_Dollar':'ASD_6p_def_Dollar', 
                                                                '6p_def_Cents':'ASD_6p_def_Cents', '3p_Dollar':'ASD_3p_Dollar','3p_Cents':'ASD_3p_Cents'})

In [428]:
spec_index = cumulative_ASD_assets[cumulative_ASD_assets['ASD_6p_def_Cents'].apply(lambda x: type(x) == str)].index
cumulative_ASD_assets.loc[spec_index, ['ASD_6p_def_Cents']] = 0

In [429]:
cumulative_ASD_assets_agg = cumulative_ASD_assets.groupby(['delegate name', 'First Name', 'Last Name',  'delegate_state', 'sign'])[['ASD_6p_Dollar','ASD_6p_Cents','ASD_6p_def_Dollar','ASD_6p_def_Cents','ASD_3p_Dollar','ASD_3p_Cents']].agg(['sum','count'])
cumulative_ASD_assets_agg.sort_values('Last Name', inplace = True)

In [430]:
cumulative_ASD_assets_agg

ASD_6p_Dollar        \
                                                                   sum count   
delegate name   First Name Last Name delegate_state sign                       
Samuel Adams    Samuel     Adams     MA             yes            0.0     0   
Matthew Adgate  Matthew    Adgate    NY             no           129.0     1   
Thomas Allen    Thomas     Allen     VA             no            12.0     1   
John  Allen     John       Allen     VA             yes           26.0     1   
John Allen      John       Allen     NC             yes           26.0     1   
...                                                                ...   ...   
John Wood       John       Wood      NY             no           160.0     1   
Joseph Wood     Joseph     Wood      MA             yes          104.0     1   
Archibald Woods Archibald  Woods     VA             yes          231.0     1   
George Wythe    George     Wythe     Constitution   no           760.0     1   
                                     VA             yes          760.0     1   

                                                         ASD_6p_Cents        \
                                                                  sum count   
delegate name   First Name Last Name delegate_state sign                      
Samuel Adams    Samuel     Adams     MA             yes           0.0     0   
Matthew Adgate  Matthew    Adgate    NY             no           39.0     1   
Thomas Allen    Thomas     Allen     VA             no           64.0     1   
John  Allen     John       Allen     VA             yes          42.0     1   
John Allen      John       Allen     NC             yes          42.0     1   
...                                                               ...   ...   
John Wood       John       Wood      NY             no           27.0     1   
Joseph Wood     Joseph     Wood      MA             yes          47.0     1   
Archibald Woods Archibald  Woods     VA             yes           3.0     1   
George Wythe    George     Wythe     Constitution   no           19.0     1   
                                     VA             yes          19.0     1   

                                                         ASD_6p_def_Dollar  \
                                                                       sum   
delegate name   First Name Last Name delegate_state sign                     
Samuel Adams    Samuel     Adams     MA             yes              231.0   
Matthew Adgate  Matthew    Adgate    NY             no                64.0   
Thomas Allen    Thomas     Allen     VA             no                 6.0   
John  Allen     John       Allen     VA             yes               13.0   
John Allen      John       Allen     NC             yes               13.0   
...                                                                    ...   
John Wood       John       Wood      NY             no                80.0   
Joseph Wood     Joseph     Wood      MA             yes               52.0   
Archibald Woods Archibald  Woods     VA             yes              115.0   
George Wythe    George     Wythe     Constitution   no               380.0   
                                     VA             yes              380.0   

                                                                \
                                                         count   
delegate name   First Name Last Name delegate_state sign         
Samuel Adams    Samuel     Adams     MA             yes      1   
Matthew Adgate  Matthew    Adgate    NY             no       1   
Thomas Allen    Thomas     Allen     VA             no       1   
John  Allen     John       Allen     VA             yes      1   
John Allen      John       Allen     NC             yes      1   
...                                                        ...   
John Wood       John       Wood      NY             no       1   
Joseph Wood     Joseph     Wood      MA             yes      1   
Archibald 

# Connecticut Continental Debt Dataset Matching

In [31]:
#prepare loan dataset
CT_CD = pd.read_excel("Data/Post1790/CT/CT_post1790_CD_ledger.xlsx", header = 13, usecols = 'H, I, N, O, X, Y, AD, AE, AN, AO, AT, AU')
CT_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
CT_CD['full name 1'] = CT_CD['First Name'] + " " + CT_CD['Last Name']
CT_CD['full name 2'] = CT_CD['First Name.1'] + " " + CT_CD['Last Name.1']
CT_CD['full name 3'] = CT_CD['First Name.2'] + " " + CT_CD['Last Name.2']

In [32]:
CT_CD['index'] = CT_CD.index

CT_CD_f1 = CT_CD[['full name 1', '6p_Dollar','6p_Cents',]]
CT_CD_f2 = CT_CD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
CT_CD_f3 = CT_CD[['full name 3', '3p_Dollar', '3p_Cents']]

In [33]:
CT_CD_6p = pd.merge(total_assets_final, CT_CD_f1[CT_CD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
CT_CD_6p_def = pd.merge(total_assets_final, CT_CD_f2[CT_CD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
CT_CD_3p = pd.merge(total_assets_final, CT_CD_f3[CT_CD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [434]:
CT_CD_agg = pd.merge(CT_CD_6p, CT_CD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
CT_CD_agg = pd.merge(CT_CD_agg, CT_CD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
CT_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Roger Sherman,NaN,yes,229.0,97.0,114.0,98.0,288.0,NaN
1,Roger Sherman,NaN,yes,229.0,97.0,114.0,98.0,10.0,4.0
2,Roger Sherman,NaN,yes,229.0,97.0,114.0,98.0,3.0,90.0
3,Roger Sherman,NaN,yes,229.0,97.0,2.0,47.0,288.0,NaN
4,Roger Sherman,NaN,yes,229.0,97.0,2.0,47.0,10.0,4.0


# Maryland Continental Debt Dataset Matching

In [435]:
#prepare loan dataset
MD_CD = pd.read_excel("Data/Post1790/MD/MD_post1790_CD.xlsx", header = 11, usecols = 'G, H, L, M, U, V, Z, AA, AI, AJ, AN, AO')
MD_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
MD_CD['full name 1'] = MD_CD['First Name'] + " " + MD_CD['Last Name']
MD_CD['full name 2'] = MD_CD['First Name.1'] + " " + MD_CD['Last Name.1']
MD_CD['full name 3'] = MD_CD['First Name.2'].apply(lambda x: str(x)) + " " + MD_CD['Last Name.2']

In [436]:
MD_CD['index'] = MD_CD.index

MD_CD_f1 = MD_CD[['full name 1', '6p_Dollar','6p_Cents',]]
MD_CD_f2 = MD_CD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
MD_CD_f3 = MD_CD[['full name 3', '3p_Dollar', '3p_Cents']]

In [437]:
MD_CD_6p = pd.merge(total_assets_final, MD_CD_f1[MD_CD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
MD_CD_6p_def = pd.merge(total_assets_final, MD_CD_f2[MD_CD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
MD_CD_3p = pd.merge(total_assets_final, MD_CD_f3[MD_CD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [438]:
MD_CD_agg = pd.merge(MD_CD_6p, MD_CD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
MD_CD_agg = pd.merge(MD_CD_agg, MD_CD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
MD_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Luther Martin,NaN,no,967.0,94.0,483.0,97.0,540.0,76.0
1,Luther Martin,NaN,no,967.0,94.0,483.0,97.0,1068.0,NaN
2,Luther Martin,MD,no,967.0,94.0,483.0,97.0,540.0,76.0
3,Luther Martin,MD,no,967.0,94.0,483.0,97.0,1068.0,NaN
4,Alexander Hamilton,NaN,yes,2822.0,88.0,1411.0,44.0,847.0,53.0


In [439]:
cumulative_CD = pd.concat([CT_CD_agg, MD_CD_agg])

# North Carolina Continental Debt Dataset Matching

In [440]:
#prepare loan dataset
NC_CD = pd.read_excel("Data/Post1790/NC/T695_R4_NC_CD.xlsx", header = 11, usecols = 'J, K, W, X, Z, AA, AC, AD ')
NC_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NC_CD['full name 1'] = NC_CD['First Name'] + " " + NC_CD['Last Name']

In [441]:
NC_CD_f = NC_CD[['full name 1', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']]

In [442]:
NC_CD_agg = pd.merge(total_assets_final, NC_CD_f[NC_CD_f['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1', 'match name'], axis = 1)

In [443]:
NC_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,John Ingram,DE,yes,81.0,80.0,40.0,89.0,NaN,NaN
1,John Ingram,NC,yes,81.0,80.0,40.0,89.0,NaN,NaN
2,Thomas Blount,NC,yes,254.0,73.0,127.0,36.0,179.0,20.0
3,John A. Campbell,NC,no,14.0,94.0,7.0,47.0,NaN,NaN


In [444]:
cumulative_CD = pd.concat([NC_CD_agg, cumulative_CD])

# New Hampshire Continental Debt Dataset Matching

In [445]:
#prepare loan dataset
NH_CD = pd.read_excel("Data/Post1790/NH/T652_R6_New_Hampshire_CD.xlsx", header = 10, usecols = 'I, J, N, O, P, Q, R, S')
NH_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',  '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NH_CD['full name 1'] = NH_CD['First Name'] + " " + NH_CD['Last Name']

In [446]:
NH_CD['index'] = NH_CD.index

NH_CD_f = NH_CD[['full name 1', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']]

In [447]:
NH_CD_agg = pd.merge(total_assets_final, NH_CD_f[NH_CD_f['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1', 'match name'], axis = 1)

In [448]:
NH_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,John Langdon,NaN,yes,12396.0,79.0,6198.0,39.0,7977.0,60.0
1,John Langdon,NH,yes,12396.0,79.0,6198.0,39.0,7977.0,60.0
2,John Parker,SC,yes,38.0,2.0,19.0,1.0,10.0,26.0
3,Jacob Brown,SC,no,582.0,35.0,291.0,18.0,218.0,85.0
4,Thomas Williams,NC,no,400.0,0.0,200.0,0.0,349.0,60.0


In [449]:
cumulative_CD = pd.concat([NH_CD_agg, cumulative_CD])

# New York Continental Debt Dataset Matching

In [450]:
#prepare loan dataset
NY_CD = pd.read_excel("Data/Post1790/NY/NY_1790_CD.xlsx", header = 11, usecols = 'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS')
NY_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
NY_CD['full name 1'] = NY_CD['First Name'] + " " + NY_CD['Last Name']
NY_CD['full name 2'] = NY_CD['First Name.1'] + " " + NY_CD['Last Name.1']
NY_CD['full name 3'] = NY_CD['First Name.2'] + " " + NY_CD['Last Name.2']

In [451]:
NY_CD['index'] = NY_CD.index

NY_CD_f1 = NY_CD[['full name 1', '6p_Dollar','6p_Cents',]]
NY_CD_f2 = NY_CD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
NY_CD_f3 = NY_CD[['full name 3', '3p_Dollar', '3p_Cents']]

In [452]:
NY_CD_6p = pd.merge(total_assets_final, NY_CD_f1[NY_CD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
NY_CD_6p_def = pd.merge(total_assets_final, NY_CD_f2[NY_CD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
NY_CD_3p = pd.merge(total_assets_final, NY_CD_f3[NY_CD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [453]:
NY_CD_agg = pd.merge(NY_CD_6p, NY_CD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
NY_CD_agg = pd.merge(NY_CD_agg, NY_CD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
NY_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Jonathan Dayton,NaN,yes,17060.0,82.0,8530.0,40.0,11332.0,93.0
1,Jonathan Dayton,NaN,yes,17060.0,82.0,8530.0,40.0,2069.0,91.0
2,Jonathan Dayton,NaN,yes,17060.0,82.0,8530.0,40.0,376.0,76.0
3,Jonathan Dayton,NaN,yes,17060.0,82.0,8530.0,40.0,2496.0,78.0
4,Jonathan Dayton,NaN,yes,17060.0,82.0,8530.0,40.0,1050.0,0.0


In [454]:
cumulative_CD = pd.concat([NY_CD_agg, cumulative_CD])

# South Carolina Continental Debt Dataset Matching

In [455]:
#prepare loan dataset
SC_CD = pd.read_excel("Data/Post1790/SC/Post_1790_South_Carolina_CD.xlsx", header = 11, usecols = 'D, E, M, N, S, T, AB, AC, AH, AI, AQ, AR')
SC_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
SC_CD['full name 1'] = SC_CD['First Name'] + " " + SC_CD['Last Name']
SC_CD['full name 2'] = SC_CD['First Name.1'] + " " + SC_CD['Last Name.1']
SC_CD['full name 3'] = SC_CD['First Name.2'].apply(lambda x: str(x)) + " " + SC_CD['Last Name.2']

In [456]:
SC_CD['index'] = SC_CD.index

SC_CD_f1 = SC_CD[['full name 1', '6p_Dollar','6p_Cents',]]
SC_CD_f2 = SC_CD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
SC_CD_f3 = SC_CD[['full name 3', '3p_Dollar', '3p_Cents']]

In [457]:
SC_CD_6p = pd.merge(total_assets_final, SC_CD_f1[SC_CD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
SC_CD_6p_def = pd.merge(total_assets_final, SC_CD_f2[SC_CD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
SC_CD_3p = pd.merge(total_assets_final, SC_CD_f3[SC_CD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [458]:
SC_CD_agg = pd.merge(SC_CD_6p, SC_CD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
SC_CD_agg = pd.merge(SC_CD_agg, SC_CD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
SC_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Daniel Hall,CT,no,123.0,29.0,61.0,65.0,NaN,NaN
1,George Parker,VA,yes,33.0,33.0,16.0,67.0,31.0,67.0
2,Thomas Smith,VA,yes,10508.0,90.0,NaN,NaN,NaN,NaN
3,Thomas Smith,MA,no,10508.0,90.0,NaN,NaN,NaN,NaN
4,William Mason,VA,yes,1024.0,91.0,512.0,46.0,276.0,65.0


In [459]:
cumulative_CD = pd.concat([cumulative_CD, SC_CD_agg])

# Pennsylvania Continental Debt Dataset Matching

In [460]:
#prepare loan dataset
PA_CD = pd.read_excel("Data/Post1790/PA/PA_post1790_CD.xlsx", header = 11, usecols = 'G, H, L, M, U, V, Z, AA, AI, AJ, AO, AP')
PA_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
PA_CD['full name 1'] = PA_CD['First Name'] + " " + PA_CD['Last Name']
PA_CD['full name 2'] = PA_CD['First Name.1'] + " " + PA_CD['Last Name.1']
PA_CD['full name 3'] = PA_CD['First Name.2'].apply(lambda x: str(x)) + " " + PA_CD['Last Name.2']

In [461]:
PA_CD['index'] = PA_CD.index

PA_CD_f1 = PA_CD[['full name 1', '6p_Dollar','6p_Cents',]]
PA_CD_f2 = PA_CD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
PA_CD_f3 = PA_CD[['full name 3', '3p_Dollar', '3p_Cents']]

In [462]:
PA_CD_6p = pd.merge(total_assets_final, PA_CD_f1[PA_CD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
PA_CD_6p_def = pd.merge(total_assets_final, PA_CD_f2[PA_CD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
PA_CD_3p = pd.merge(total_assets_final, PA_CD_f3[PA_CD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [463]:
PA_CD_agg = pd.merge(PA_CD_6p, PA_CD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
PA_CD_agg = pd.merge(PA_CD_agg, PA_CD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
PA_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Daniel Carroll,NaN,yes,151.0,4.0,75.0,52.0,NaN,NaN
1,Elbridge Gerry,NaN,no,1516.0,67.0,758.0,33.0,NaN,NaN
2,Hugh Williamson,NaN,yes,149.0,98.0,75.0,NaN,NaN,NaN
3,Hugh Williamson,NC,yes,149.0,98.0,75.0,NaN,NaN,NaN
4,Robert Morris,NaN,yes,2390.0,36.0,1195.0,18.0,NaN,NaN


In [464]:
cumulative_CD = pd.concat([cumulative_CD, PA_CD_agg])

# Rhode Island Continental Debt Dataset Matching

In [465]:
#prepare loan dataset
RI_CD = pd.read_excel("Data/Post1790/RI/T653_Rhode_Island_CD.xlsx", header = 11, usecols = 'G, H, L, M, U, V, Z, AA, AI, AJ, AN, AO')
RI_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
RI_CD['full name 1'] = RI_CD['First Name'] + " " + RI_CD['Last Name']
RI_CD['full name 2'] = RI_CD['First Name.1'] + " " + RI_CD['Last Name.1']
RI_CD['full name 3'] = RI_CD['First Name.2'] + " " + RI_CD['Last Name.2']

In [466]:
RI_CD['index'] = RI_CD.index

RI_CD_f1 = RI_CD[['full name 1', '6p_Dollar','6p_Cents',]]
RI_CD_f2 = RI_CD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
RI_CD_f3 = RI_CD[['full name 3', '3p_Dollar', '3p_Cents']]

In [467]:
RI_CD_6p = pd.merge(total_assets_final, RI_CD_f1[RI_CD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
RI_CD_6p_def = pd.merge(total_assets_final, RI_CD_f2[RI_CD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
RI_CD_3p = pd.merge(total_assets_final, RI_CD_f3[RI_CD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [468]:
RI_CD_agg = pd.merge(RI_CD_6p, RI_CD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
RI_CD_agg = pd.merge(RI_CD_agg, RI_CD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
RI_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Luther Martin,NaN,no,20.0,63.0,10.0,31.0,14.0,73.0
1,Luther Martin,MD,no,20.0,63.0,10.0,31.0,14.0,73.0
2,William Hall,DE,yes,167.0,98.0,83.0,98.0,87.0,41.0
3,Richard Smith,DE,yes,43.0,45.0,21.0,72.0,40.0,97.0
4,Joseph Barker,DE,yes,58.0,64.0,29.0,32.0,21.0,12.0


In [469]:
cumulative_CD = pd.concat([RI_CD_agg, cumulative_CD])

# Virginia Continental Debt Dataset Matching

In [470]:
#prepare loan dataset
VA_CD = pd.read_excel("Data/Post1790/VA/VA_CD.xlsx", header = 11, usecols = 'H, I, K, L, U, V, X, Y, AH, AI, AK, AL')
VA_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
VA_CD['full name 1'] = VA_CD['First Name'] + " " + VA_CD['Last Name']
VA_CD['full name 2'] = VA_CD['First Name.1'] + " " + VA_CD['Last Name.1']
VA_CD['full name 3'] = VA_CD['First Name.2'] + " " + VA_CD['Last Name.2']

In [471]:
VA_CD['index'] = VA_CD.index

VA_CD_f1 = VA_CD[['full name 1', '6p_Dollar','6p_Cents',]]
VA_CD_f2 = VA_CD[['full name 2', '6p_def_Dollar','6p_def_Cents']]
VA_CD_f3 = VA_CD[['full name 3', '3p_Dollar', '3p_Cents']]

In [472]:
VA_CD_6p = pd.merge(total_assets_final, VA_CD_f1[VA_CD_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1','match name'], axis = 1)
VA_CD_6p_def = pd.merge(total_assets_final, VA_CD_f2[VA_CD_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop(['full name 2','match name'], axis = 1)
VA_CD_3p = pd.merge(total_assets_final, VA_CD_f3[VA_CD_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop(['full name 3','match name'], axis = 1)

In [473]:
VA_CD_agg = pd.merge(VA_CD_6p, VA_CD_6p_def, on = ['delegate name','delegate_state','sign'], how = 'outer')
VA_CD_agg = pd.merge(VA_CD_agg, VA_CD_3p, on = ['delegate name','delegate_state','sign'], how = 'outer')
VA_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,George Read,NaN,yes,767.0,88.0,NaN,NaN,NaN,NaN
1,Robert Morris,NaN,yes,5000.0,0.0,NaN,NaN,NaN,NaN
2,John Blair,NaN,yes,652.0,86.0,326.0,44.0,386.0,53.0
3,John Blair,NaN,yes,652.0,86.0,326.0,44.0,242.0,75.0
4,John Blair,NaN,yes,652.0,86.0,326.0,44.0,25.0,NaN


In [474]:
cumulative_CD = pd.concat([VA_CD_agg, cumulative_CD])

# Georgia Continental Debt Dataset Matching

In [475]:
#prepare loan dataset
GA_CD = pd.read_excel("Data/Post1790/GA/T694_GA_Loan_Office_CD.xlsx", header = 10, usecols = 'Q, R, Z, AA, AB, AC, AD, AE')
GA_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',  '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
GA_CD['full name 1'] = GA_CD['First Name'] + " " + GA_CD['Last Name']

In [476]:
GA_CD['index'] = GA_CD.index

GA_CD_f = GA_CD[['full name 1', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']]

In [477]:
GA_CD_agg = pd.merge(total_assets_final, GA_CD_f[GA_CD_f['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1', 'match name'], axis = 1)

In [478]:
GA_CD_agg.head()

,delegate name,delegate_state,sign,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents
0,Nathaniel Russell,SC,yes,1.0,3987.9033,1.0,1993.9533,1.0,3625.67
1,Henry Laurens,SC,yes,1.0,824.4133,1.0,412.2033,1.0,941.54
2,Henry Laurens,SC,yes,1.0,824.4133,1.0,412.2033,1.0,941.54


In [479]:
cumulative_CD = pd.concat([GA_CD_agg, cumulative_CD])

# New Jersey Continental Debt Dataset Matching

In [1]:
#prepare loan dataset
NJ_CD = pd.read_excel("Data/Post1790/NJ/NJ_3_percent_stock_T698_R1_R2.xlsx", header = 10, usecols = 'D, E, L, M')
NJ_CD.columns = ['First Name', 'Last Name', '3p_Dollar', '3p_Cents']
NJ_CD['full name 1'] = NJ_CD['First Name'] + " " + NJ_CD['Last Name']

<IPython.core.display.Javascript object>

In [2]:
NJ_CD['index'] = NJ_CD.index

NJ_CD_f = NJ_CD[['full name 1',  '3p_Dollar', '3p_Cents']]

In [482]:
NJ_CD_agg = pd.merge(total_assets_final, NJ_CD_f[NJ_CD_f['full name 1'].apply(lambda x: not pd.isnull(x))],
                     left_on = 'match name', right_on = 'full name 1').drop(['full name 1', 'match name'], axis = 1)

In [483]:
NJ_CD_agg.head()

,delegate name,delegate_state,sign,3p_Dollar,3p_Cents
0,Jonathan Dayton,NaN,yes,168,52
1,Jonathan Dayton,NaN,yes,302,9
2,Robert Morris,NaN,yes,156,53
3,Robert Morris,NaN,yes,69,9
4,Robert Morris,NaN,yes,166,15


In [484]:
cumulative_CD = pd.concat([NJ_CD_agg, cumulative_CD])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


# Assembling the Continental Debt File

In [485]:
cumulative_CD['delegate_state'] = cumulative_CD['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

In [486]:
cumulative_CD_assets = pd.merge(cum_firstlast, cumulative_CD, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
cumulative_CD_assets = cumulative_CD_assets.rename(columns = {'6p_Dollar':'CD_6p_Dollar', '6p_Cents':'CD_6p_Cents','6p_def_Dollar':'CD_6p_def_Dollar', 
                                                                '6p_def_Cents':'CD_6p_def_Cents', '3p_Dollar':'CD_3p_Dollar','3p_Cents':'CD_3p_Cents'})

In [487]:
cumulative_CD_assets_agg = cumulative_CD_assets.groupby(['delegate name', 'First Name', 'Last Name',  'delegate_state', 'sign'])[['CD_6p_Dollar','CD_6p_Cents','CD_6p_def_Dollar','CD_6p_def_Cents','CD_3p_Dollar','CD_3p_Cents']].agg(['sum','count'])
cumulative_CD_assets_agg.sort_values('Last Name', inplace = True)

In [488]:
CD_ASD_merge = pd.merge(cumulative_CD_assets_agg, cumulative_ASD_assets_agg, on = ['delegate name','First Name','Last Name','delegate_state','sign'], how = 'outer')
CD_ASD_merge

CD_6p_Dollar        \
                                                                  sum count   
delegate name   First Name Last Name delegate_state sign                      
John Allen      John       Allen     NC             yes         116.0   1.0   
John  Allen     John       Allen     VA             yes         116.0   1.0   
Thomas Allen    Thomas     Allen     VA             no          357.0   1.0   
Thomas Anderson Thomas     Anderson  NJ             yes           0.0   0.0   
James Anderson  James      Anderson  NC             yes          32.0   1.0   
...                                                               ...   ...   
Joseph Wood     Joseph     Wood      MA             no            NaN   NaN   
                                                    yes           NaN   NaN   
Archibald Woods Archibald  Woods     VA             yes           NaN   NaN   
George Wythe    George     Wythe     Constitution   no            NaN   NaN   
                                     VA             yes           NaN   NaN   

                                                         CD_6p_Cents        \
                                                                 sum count   
delegate name   First Name Last Name delegate_state sign                     
John Allen      John       Allen     NC             yes         31.0   1.0   
John  Allen     John       Allen     VA             yes         31.0   1.0   
Thomas Allen    Thomas     Allen     VA             no          38.0   1.0   
Thomas Anderson Thomas     Anderson  NJ             yes          0.0   0.0   
James Anderson  James      Anderson  NC             yes         48.0   1.0   
...                                                              ...   ...   
Joseph Wood     Joseph     Wood      MA             no           NaN   NaN   
                                                    yes          NaN   NaN   
Archibald Woods Archibald  Woods     VA             yes          NaN   NaN   
George Wythe    George     Wythe     Constitution   no           NaN   NaN   
                                     VA             yes          NaN   NaN   

                                                         CD_6p_def_Dollar  \
                                                                      sum   
delegate name   First Name Last Name delegate_state sign                    
John Allen      John       Allen     NC             yes              58.0   
John  Allen     John       Allen     VA             yes              58.0   
Thomas Allen    Thomas     Allen     VA             no              178.0   
Thomas Anderson Thomas     Anderson  NJ             yes               0.0   
James Anderson  James      Anderson  NC             yes              16.0   
...                                                                   ...   
Joseph Wood     Joseph     Wood      MA             no                NaN   
                                                    yes               NaN   
Archibald Woods Archibald  Woods     VA             yes               NaN   
George Wythe    George     Wythe     Constitution   no                NaN   
                                     VA             yes               NaN   

                                                                \
                                                         count   
delegate name   First Name Last Name delegate_state sign         
John Allen      John       Allen     NC             yes    1.0   
John  Allen     John       Allen     VA             yes    1.0   
Thomas Allen    Thomas     Allen     VA             no     1.0   
Thomas Anderson Thomas     Anderson  NJ             yes    0.0   
James Anderson  James      Anderson  NC             yes    1.0   
...                                                        ...   
Joseph Wood     Joseph     Wood      MA             no     NaN   
                                                    yes    NaN   
Archibald Woods Archibald  Woods     VA             

In [489]:
cum_delegate_list = pd.merge(total_assets, cum_firstlast, left_on = 'delegate name', right_on = 'Full Name').drop_duplicates().drop(['match name', 'Full Name'], axis = 1)
spec_index = cum_delegate_list[cum_delegate_list['delegate_state'].apply(lambda x: pd.isnull(x))].index
cum_delegate_list.loc[spec_index, ['delegate_state']] = 'Constitution'
cum_delegate_list = cum_delegate_list[['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign']]

In [490]:
for row in cum_delegate_list.index:
    row_vals = tuple(cum_delegate_list.loc[row].tolist())
    if row_vals not in CD_ASD_merge.index:
        CD_ASD_merge.ix[row_vals,:] = [np.nan] * 24

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [491]:
CD_ASD_merge = CD_ASD_merge.sort_values('Last Name')

In [492]:
CD_ASD_merge.to_csv("Producables/Delegates_Post1790_Assets.csv")

In [493]:
cumulative_CD_assets_agg.to_csv('Producables/Separate/CD_debt_Matched.csv')

In [494]:
cumulative_ASD_assets_agg.to_csv('Producables/Separate/ASD_debt_Matched.csv')